# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек,-настройка,-константы" data-toc-modified-id="Импорт-библиотек,-настройка,-константы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек, настройка, константы</a></span></li><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#Подготовка-эмбеддингов" data-toc-modified-id="Подготовка-эмбеддингов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Подготовка эмбеддингов</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TF-IDF</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Catboost</a></span></li></ul></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>BERT</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия-c-BERT" data-toc-modified-id="Логистическая-регрессия-c-BERT-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Логистическая регрессия c BERT</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование-моделей." data-toc-modified-id="Тестирование-моделей.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование моделей.</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

### Импорт библиотек, настройка, константы

In [1]:
!pip install torch
!pip install transformers
!pip install catboost
!pip install warnings

ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers
import warnings

from tqdm import notebook
from tqdm.notebook import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

from catboost import CatBoostClassifier

import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
 
tqdm.pandas()

pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")
RANDOM_STATE = 12345

#Максимальное количество эмбеддингов, и размер батча для их получения.
BERT_SAMPLES = 2000
BATCH_SIZE = 50

#Путь к датасету
PATH_TO_DATASET = 'C:\\Users\\Ivan\\Downloads\\Yandex_Practicum\\text\\toxic_comments.csv'

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузка датасета

Мы будем решать задачу двумя способами, 'классичесским' через TF-IDF и с помощью эмбеддингов предобученной модели *BERT*.    

Выделим для последней сразу выборку. Так как модель *BERT* достаточно тяжёлая, и эмбединги создаются долго, попробуем ограничиться выборкой в 2000 строк (500 строк - запас для дальнейшей фильтрации последовательностей токенов > 512).

In [3]:
df = pd.read_csv(PATH_TO_DATASET)
df_bert = df.copy()
df_bert = df_bert.sample(BERT_SAMPLES + 500, random_state = RANDOM_STATE).reset_index(drop = True) 
df_bert.head()

,text,toxic
0,Ahh shut the fuck up you douchebag sand nigger \n\nGo blow up some more people you muslim piece of shit. Fuck you sand nigger i will find u in real life and slit your throat.,1
1,"""\n\nREPLY: There is no such thing as Texas Commerce Bank of Chicago. Likewise, there is no such thing as the United Farmers Bank of Baltimore and Albuquerque. So Salvio, you are incorrect. If you want to prevent even the remote possibility of confusion, then you should not be allowed to use your name, Salvio, because there may be confusion that you are related to Salvador Dali.\n\n""",0
2,"Reply\nHey, you could at least mention Jasenovac and 700 000 killed (not only serbs) but you say it's all bs, well, what is vandalism, death of innocent, or putting truth here?",0
3,"Thats fine, there is no deadline ) chi?",0
4,"""\n\nDYK nomination of Mustarabim\n Hello! Your submission of Mustarabim at the Did You Know nominations page has been reviewed, and there still are some issues that may need to be clarified. Please review the comment(s) underneath and respond there as soon as possible. Thank you for contributing to Did You Know! (talk • contribs) """,0


### TF-IDF

In [4]:
lemmatizer = WordNetLemmatizer()

Напишем функции лемматизации и очистки текста.

In [5]:
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])  
    return lemmatized_output

def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.split()) 

Применим функции к нашему тексту.

In [6]:
df['text_lemma'] = df['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

  0%|          | 0/159571 [00:00<?, ?it/s]

Я не знаю как делать 'честную' кросс-валидацию не залезая в обучающую выборку при создании векторов, поэтому разделим данные на обучающую/валидационную и тестовую выборку в соотношении 3:1:1

In [7]:
corpus_all = df['text_lemma'].values.astype('U')
target_all = df['toxic']

corpus, corpus_test, target, target_test = train_test_split(
    corpus_all, target_all, test_size=0.2, random_state = RANDOM_STATE)

corpus_train, corpus_valid, target_train, target_valid = train_test_split(
    corpus, target, test_size = 0.25, random_state = RANDOM_STATE)

In [8]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords) 
tf_idf = count_tf_idf.fit_transform(corpus_train) 
print("Размер матрицы tf-idf:", tf_idf.shape)

Размер матрицы tf-idf: (95742, 121091)


In [9]:
tf_idf_test = count_tf_idf.transform(corpus_test) 
print("Размер матрицы tf-idf_test:", tf_idf_test.shape)

Размер матрицы tf-idf_test: (31915, 121091)


In [10]:
tf_idf_valid = count_tf_idf.transform(corpus_valid) 
print("Размер матрицы tf-idf_valid:", tf_idf_valid.shape)

Размер матрицы tf-idf_valid: (31914, 121091)


Корпуса готовы, размерности в порядке, обучение во второй главе.

### Подготовка эмбеддингов

Загрузим пре-тренированные токенайзер и модель.

In [11]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токенизируем все комментарии.

In [12]:
tokenized = df_bert['text'].progress_apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

  0%|          | 0/2500 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors


Для корректной работы модели последовательность токенов (и маска) должны быть одной длины. Дополним нулями и создадим маску важных токенов.   
Также заметим, что у модели *BERT* есть ограничение на длину последовательности токенов в 512, иначе, в дальнейшем, будет происходить ошибка индексов. Поэтому отберём только те примеры, где длина меньше 512.

In [13]:
tokens = []
target = []
for i in range(len(tokenized)):
    if len(tokenized[i]) <= 512:
        tokens.append(tokenized[i])
        target.append(df_bert['toxic'][i])
tokens = (pd.Series(tokens)).head(BERT_SAMPLES)
target = (pd.Series(target)).head(BERT_SAMPLES)

In [14]:
max_len = 0
for i in tqdm(tokens.values):
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tqdm(tokens.values)])
attention_mask = np.where(padded != 0, 1, 0)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [15]:
display(padded.shape, attention_mask.shape)

(2000, 511)

(2000, 511)

Чтобы не забить всю память разом до отказа эмбеддинги будем создавать порционно, радуясь перемещению полоски.

In [16]:
batch_size = BATCH_SIZE
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
features_bert = np.concatenate(embeddings)

  0%|          | 0/40 [00:00<?, ?it/s]

**Вывод 1-го шага:**  
+ Мы загрузили данные, представляющие собой комментарии с разметкой о токсичности.
+ Подготовили как TF-IDF признаки, так и ембеддинги с помощью модели *BERT*.
+ Данные готовы к обучению.

## Обучение

Напишем функцию обучения моделей и вычисления метрик. Она годится как для TF-IDF подхода, так и для эмбеддингов.

In [17]:
scores = pd.DataFrame()
def fit_predict_cv(model, features, target, scores, features_valid = None, target_valid = None, cv = None):
    model = model
    model.fit(features, target)
    F1_valid, F1_cv = None, None
    
    F1_train = f1_score(target, model.predict(features))
    
    if cv == None:
        F1_valid = f1_score(target_valid, model.predict(features_valid))
    else: 
        F1_cv = cross_val_score(model, features, target, cv = cv, scoring = 'f1').mean() 
    
    scores = scores.append({'model' : type(model).__name__,
                            'F1_train' : F1_train,
                            'F1_valid' : F1_valid,
                            'F1_train_cv' : F1_cv} , ignore_index = True)
    return scores

### TF-IDF

#### Логистическая регрессия

Т.к. выборка несбалансированная по целевому признаку, поставим `class_weight = balanced`.

In [18]:
scores = fit_predict_cv(LogisticRegression(class_weight = 'balanced'), 
                        tf_idf, target_train, scores = scores, features_valid = tf_idf_valid, target_valid = target_valid)
scores

,F1_train,F1_train_cv,F1_valid,model
0,0.836801,None,0.744242,LogisticRegression


#### Catboost

In [19]:
scores = fit_predict_cv(CatBoostClassifier(
    iterations = 200, learning_rate = 0.5, eval_metric = 'F1', verbose = 20, random_state = RANDOM_STATE), 
                        tf_idf, target_train, scores = scores, features_valid = tf_idf_valid, target_valid = target_valid)
scores

0:	learn: 0.4089165	total: 748ms	remaining: 2m 28s
20:	learn: 0.6766365	total: 11.6s	remaining: 1m 38s
40:	learn: 0.7256002	total: 22.4s	remaining: 1m 26s
60:	learn: 0.7530917	total: 33.2s	remaining: 1m 15s
80:	learn: 0.7686712	total: 43.8s	remaining: 1m 4s
100:	learn: 0.7804438	total: 54.5s	remaining: 53.4s
120:	learn: 0.7866204	total: 1m 5s	remaining: 42.5s
140:	learn: 0.7987213	total: 1m 15s	remaining: 31.8s
160:	learn: 0.8069577	total: 1m 26s	remaining: 21s
180:	learn: 0.8155759	total: 1m 37s	remaining: 10.2s
199:	learn: 0.8222972	total: 1m 47s	remaining: 0us


,F1_train,F1_train_cv,F1_valid,model
0,0.836801,None,0.744242,LogisticRegression
1,0.822297,None,0.764283,CatBoostClassifier


### BERT

Разделим данные на обучающую и тестовую выборки в соотношении 75:25. Будем использовать кросс-валидацию.

In [20]:
target_bert = target.head(len(features_bert))
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(
    features_bert, target_bert, test_size = 0.25, random_state = RANDOM_STATE)

#### Логистическая регрессия c BERT

При количестве итераций по умолчанию (100) модель плохо сходится, увеличим это значение до `max_iter = 1000`.

In [21]:
scores = fit_predict_cv(LogisticRegression(max_iter = 1000), features_train_bert, target_train_bert, scores, cv = 5)
scores

,F1_train,F1_train_cv,F1_valid,model
0,0.836801,None,0.744242,LogisticRegression
1,0.822297,None,0.764283,CatBoostClassifier
2,0.970588,0.678365,NaN,LogisticRegression


**Вывод 2-го шага:**

+ Мы обучили модели для предсказания токсичности комментариев, основываясь на двух подходах. Заметим, что не очень корректно сравнивать метрки между собой, т.к. для эмбедингов мы использовали всего лишь 2000 сэмплов. Однако, очевидно, преимущество в скорости TF-IDF подхода.
+ Для подхода TF-IDF наилучшей оказалась модель catboost, со значением F1 на валидационной выборке **0.76**. Порог в 0.75 преодолен.
+ Для подхода, основанном на эмбеддингах (для 2000 сэмплов) значение F1 на кросс-валидации составило **0.68**.
+ Принимая во внимание имеющиеся ресурсы примем модель `catboost` подхода *TF-IDF* за лучшую модель. Протестируем ее в следующем пункте.

## Тестирование моделей.

Посмотрим качество лучшей модели на тестовых данных.

In [22]:
model = CatBoostClassifier(iterations = 200, learning_rate = 0.5, eval_metric = 'F1', verbose = 20, random_state = RANDOM_STATE)
model.fit(tf_idf, target_train)
print('F1 мера лучшей модели на тестовых данных:', f1_score(target_test, model.predict(tf_idf_test)))

0:	learn: 0.4089165	total: 576ms	remaining: 1m 54s
20:	learn: 0.6766365	total: 11.5s	remaining: 1m 38s
40:	learn: 0.7256002	total: 22.3s	remaining: 1m 26s
60:	learn: 0.7530917	total: 33.1s	remaining: 1m 15s
80:	learn: 0.7686712	total: 43.9s	remaining: 1m 4s
100:	learn: 0.7804438	total: 54.6s	remaining: 53.5s
120:	learn: 0.7866204	total: 1m 5s	remaining: 42.7s
140:	learn: 0.7987213	total: 1m 16s	remaining: 31.9s
160:	learn: 0.8069577	total: 1m 26s	remaining: 21.1s
180:	learn: 0.8155759	total: 1m 37s	remaining: 10.3s
199:	learn: 0.8222972	total: 1m 47s	remaining: 0us
F1 мера лучшей модели на тестовых данных: 0.7521186440677966


**Вывод:**
F1-мера для лучшей модели на тестовых данных: 0.75. Порог в 0.75 достигнут.

## Выводы

+ Мы загрузили данные, представляющие собой ~160 тыс. комментариев интернет-магазин «Викишоп». 
+ Задача решалась двумя подходами:   
    1. 'классический' **TF-IDF**  
    2. С помощью эмбеддингов, полученных на предобученной модели **BERT**
+ Мы подготовили признаки для двух подходов, для подхода на основе эмбеддингов использовали только 2000 сэмплов в целях экономии ресурсов.
+ Для подхода *TF-IDF* лучшей моделью оказалась модель *catboost*, со значением F1-меры на валидациооной выборке в **0.76**, на тестовой - **0.75**. Порог задачи в 0.75 преодолен.
+ Для подхода, основанном на эмбеддингах значение F1-меры на кросс-валидации составило **0.68** с использованием всего 2000 сэмплов, что довольно неплохо. 